# SDA - lecture 4 - Single spike train

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(asctime)s: %(message)s')
import warnings

import math
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

%matplotlib inline

In [ ]:
# Generate a "spike train" of a neuron 
samp = 1000
rate = 20 / samp
duration = 600

spk_array = (np.random.uniform(size=samp*duration)<rate).astype(np.int32)
time_array = np.arange(0, duration, 1/samp)

### The inter-spike interval (ISI) histogram

In [ ]:
spk_times = np.flatnonzero(spk_array) * 1000 / samp  # Switch from bins to milliseconds
spk_intervals = np.diff(spk_times) 

fig, ax = plt.subplots(figsize=(8,4), nrows=1, ncols=2)

np.seterr(divide = 'ignore')
hist_vals, hist_bins = np.histogram(spk_intervals, bins=np.arange(1,400,5), density=True)

ax[0].plot(hist_bins[:-1], hist_vals)
ax[0].set_xlabel('Interval [ms]')
ax[0].set_ylabel('Probability')
ax[0].set_title(f'Linear TIH')
ax[1].plot(hist_bins[:-1], np.log(hist_vals))
ax[1].set_xlabel('Interval [ms]')
ax[1].set_ylabel('Log Probability')
ax[1].set_title(f'Logarithmic TIH')

reg = LinearRegression()  
reg.fit(hist_bins[:20].reshape((-1,1)), np.log(hist_vals[:20]).reshape((-1,1)))
pred_vals = reg.predict(hist_bins.reshape((-1,1)))  
ax[1].plot(hist_bins, pred_vals.reshape(-1),'k:')

logging.info(f'Slope reflects rate {-reg.coef_[0][0]*1000:.2f} spikes/s')
logging.info(f'Interception reflects rate {np.exp(reg.intercept_[0])*1000:.2f} spikes/s')